In [2]:
!pip install transformers datasets -q

In [3]:
!pip install gdown -q
!gdown 1zqzrHcpjqYl6Gkdgj9AGGyVlBbTL93pV

Downloading...
From: https://drive.google.com/uc?id=1zqzrHcpjqYl6Gkdgj9AGGyVlBbTL93pV
To: /kaggle/working/data.xlsx
100%|█████████████████████████████████████████| 713k/713k [00:00<00:00, 116MB/s]


In [4]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset('stereoset', 'intersentence')
stereo_sentence = []
ster = dataset['validation'][0:]['bias_type']

for i in range(len(ster)):
    if ster[i] == 'gender':
        sent = dataset['validation'][i:i+1]['sentences'][0]['sentence']
        gold_label = dataset['validation'][i:i+1]['sentences'][0]['gold_label']
        for j in range(len(gold_label)):
            if gold_label[j] == 0:
                stereo_sentence.append(sent[j])

stereo_sent1 = []
ster = dataset['validation'][0:]['bias_type']

for i in range(len(ster)):
    if ster[i] == 'gender':
        sent = dataset['validation'][i:i+1]['sentences'][0]['sentence']
        gold_label = dataset['validation'][i:i+1]['sentences'][0]['gold_label']
        for j in range(len(gold_label)):
            if gold_label[j] == 1:
                #print(sent[j])
                stereo_sent1.append(sent[j])

stereo_sentence.append(stereo_sent1)

df1 = pd.DataFrame(stereo_sentence)
df2 = pd.Series([1 for x in range(len(df1.index))])
stereo_df = pd.concat([df1,df2],axis=1,ignore_index=False)
stereo_df.columns = ['data','stereotype']
string_list = stereo_df.loc[242, 'data']

# Remove the 242nd element from the DataFrame
stereo_df = stereo_df.drop(242)

# Convert the string_list into a list of dictionaries
list_of_dicts = [{'data': item, 'stereotype': 1} for item in string_list]

# Create a new DataFrame from the list of dictionaries
new_data_df = pd.DataFrame(list_of_dicts)

# Concatenate the new DataFrame with the original DataFrame
stereo_df = pd.concat([stereo_df, new_data_df], ignore_index=True)


Generating validation split:   0%|          | 0/2123 [00:00<?, ? examples/s]

Dataset stereoset downloaded and prepared to /root/.cache/huggingface/datasets/stereoset/intersentence/1.0.0/b188e395e95b37c7a095ebc2de352fbdb249d67d1beb2ff639bb4dc37dfbb090. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
stereo_df.shape

(484, 2)

In [6]:
import string

# Define a function to clean text
def clean_text(text):
    # Remove punctuation and convert to lower case
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()
    return text

In [7]:
import pandas as pd
import numpy as np

poems_df = pd.read_excel('/kaggle/working/data.xlsx', sheet_name='poems')
# Read the 'rhymes' sheet from the Excel file
rhymes_df = pd.read_excel('/kaggle/working/data.xlsx', sheet_name='rhymes')
#poems_df['poem'] = poems_df['poem']+341

## Rhymes and Poems DataFrame

In [8]:
# Clean the 'data' column in poems_df
poems_df['data'] = poems_df['data'].apply(clean_text)

# Clean the 'aug_data' column in poems_df
poems_df['aug_data'] = poems_df['aug_data'].apply(clean_text)

# Clean the 'data' column in rhymes_df
rhymes_df['data'] = rhymes_df['data'].apply(clean_text)

# Clean the 'aug_data' column in rhymes_df
rhymes_df['aug_data'] = rhymes_df['aug_data'].apply(clean_text)

stereo_df['data'] = stereo_df['data'].apply(clean_text)

## Line By Line

In [9]:
# Combine both dataframes
line_df = pd.concat([rhymes_df, poems_df],ignore_index=True)

# Add the last 'poem' number from rhymes_df to the 'poem' column of poems_df
last_poem_number = rhymes_df['poem'].max()
poems_df['poem'] += last_poem_number 

line_df['stereotype'].sum()

510

In [10]:
line_df.shape

(13313, 4)

## 2 Lines

In [11]:
combined_df = pd.concat([rhymes_df, poems_df],ignore_index=True)

# Add the last 'poem' number from rhymes_df to the 'poem' column of poems_df
last_poem_number = rhymes_df['poem'].max()
poems_df['poem'] += last_poem_number 

# Grouping poems in the combined dataframe by 2 lines
grouped_2lines_df = combined_df.groupby(['poem', np.arange(len(combined_df)) // 2]).agg({'data': ' '.join, 'aug_data': ' '.join})

# Check if a stereotype exists in the group and assign 'stereotype' column
grouped_2lines_df['stereotype'] = combined_df.groupby(['poem', np.arange(len(combined_df)) // 2])['stereotype'].apply(lambda x: 1 if x.astype(bool).any() else np.nan)

# Replace NaN values in 'stereotype' column with 0
grouped_2lines_df['stereotype'] = grouped_2lines_df['stereotype'].fillna(0).astype(int)

# Reset the index to remove the 'level_1' column
grouped_2lines_df = grouped_2lines_df.reset_index()

grouped_2lines_df['stereotype'].sum()

421

## 3 Lines

In [12]:
combined_df = pd.concat([rhymes_df, poems_df],ignore_index=True)

# Add the last 'poem' number from rhymes_df to the 'poem' column of poems_df
last_poem_number = rhymes_df['poem'].max()
poems_df['poem'] += last_poem_number 

# Grouping poems in the combined dataframe by 3 lines
grouped_3lines_df = combined_df.groupby(['poem', np.arange(len(combined_df)) // 3]).agg({'data': ' '.join, 'aug_data': ' '.join})

# Check if a stereotype exists in the group and assign 'stereotype' column
grouped_3lines_df['stereotype'] = combined_df.groupby(['poem', np.arange(len(combined_df)) // 3])['stereotype'].apply(lambda x: 1 if x.astype(bool).any() else np.nan)

# Replace NaN values in 'stereotype' column with 0
grouped_3lines_df['stereotype'] = grouped_3lines_df['stereotype'].fillna(0).astype(int)

# Reset the index to remove the 'level_1' column
grouped_3lines_df = grouped_3lines_df.reset_index()

grouped_3lines_df['stereotype'].sum()

375

## Poem by Poem

In [13]:
# Combine both dataframes
combined_df = pd.concat([rhymes_df, poems_df],ignore_index=True)

# Add the last 'poem' number from rhymes_df to the 'poem' column of poems_df
last_poem_number = rhymes_df['poem'].max()
poems_df['poem'] += last_poem_number 

# Grouping poems in the combined dataframe, treating each poem as a separate group
grouped_poem_df = combined_df.groupby('poem').agg({'data': ' '.join, 'aug_data': ' '.join})

# Check if a stereotype exists in the group and assign 'stereotype' column
grouped_poem_df['stereotype'] = combined_df.groupby('poem')['stereotype'].apply(lambda x: 1 if x.astype(bool).any() else np.nan)

# Replace NaN values in 'stereotype' column with 0
grouped_poem_df['stereotype'] = grouped_poem_df['stereotype'].fillna(0).astype(int)

# Reset the index to get 'poem' as a column
grouped_poem_df = grouped_poem_df.reset_index()

grouped_poem_df['stereotype'].sum()

145

In [14]:
# Create a list of dataframes
dataframes_list = [line_df, grouped_2lines_df, grouped_3lines_df, grouped_poem_df]

In [15]:
print('\t 0\t 1')
print('1 line\t',dataframes_list[0].shape[0] - dataframes_list[0]['stereotype'].sum() ,'\t',dataframes_list[0]['stereotype'].sum())
print('2 lines\t',dataframes_list[1].shape[0] - dataframes_list[1]['stereotype'].sum() ,'\t',dataframes_list[1]['stereotype'].sum())
print('3 lines\t',dataframes_list[2].shape[0] - dataframes_list[2]['stereotype'].sum() ,'\t',dataframes_list[2]['stereotype'].sum())
print('Poem \t',dataframes_list[3].shape[0] - dataframes_list[3]['stereotype'].sum() ,'\t',dataframes_list[3]['stereotype'].sum())

	 0	 1
1 line	 12803 	 510
2 lines	 6548 	 421
3 lines	 4516 	 375
Poem 	 516 	 145


In [16]:
stereo_df['stereotype'].sum()

484

# Function to create Different types in data

In [16]:
!pip install imbalanced-learn -q

In [17]:
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split

def pre_processing(df, stereo):
    X_train, X_test, y_train, y_test = train_test_split(df.drop(['stereotype'], axis=1), df["stereotype"], test_size=0.30, random_state=42)

    # Split the dataset into 80:20 ratio
    train_df = pd.concat([X_train, y_train], axis=1)
    print('training Sterotypes\n',train_df['stereotype'].value_counts())
    test_df = pd.concat([X_test, y_test], axis=1)

    new_df = train_df[train_df['stereotype'] == 1]
    print(len(new_df['aug_data'].tolist()))
    temp = new_df['aug_data'].tolist()
    train_df = pd.concat([train_df, pd.DataFrame({'data': temp, 'stereotype': [1] * len(temp)}), stereo], ignore_index=True)

    train_df['call_set'] = 'train'
    test_df['call_set'] = 'test'

    combined_df = pd.concat([train_df, test_df], ignore_index=True)
    combined_df.drop(['aug_data'], axis=1, inplace=True)

    return combined_df


In [19]:
line_line = pre_processing(dataframes_list[0], stereo_df).drop(['poem'], axis=1)
line_2 = pre_processing(dataframes_list[1], stereo_df).drop(['poem', 'level_1'], axis=1)
line_3 = pre_processing(dataframes_list[2], stereo_df).drop(['poem', 'level_1'], axis=1)
poem_poem = pre_processing(dataframes_list[3], stereo_df).drop(['poem'], axis=1)


training Sterotypes
 0    8966
1     353
Name: stereotype, dtype: int64
353
training Sterotypes
 0    4596
1     282
Name: stereotype, dtype: int64
282
training Sterotypes
 0    3160
1     263
Name: stereotype, dtype: int64
263
training Sterotypes
 0    359
1    103
Name: stereotype, dtype: int64
103


In [20]:
split_data_df = [line_line, line_2, line_3, poem_poem]

In [21]:
[value_counts for i in range(len(split_data_df)) for value_counts in [split_data_df[i]['stereotype'].value_counts()]]

[0    12803
 1     1347
 Name: stereotype, dtype: int64,
 0    6548
 1    1187
 Name: stereotype, dtype: int64,
 0    4516
 1    1122
 Name: stereotype, dtype: int64,
 1    732
 0    516
 Name: stereotype, dtype: int64]

___

# BERT Classification

In [22]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [23]:
# Set device to GPU if available


In [23]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [28]:
from tqdm import tqdm

def train(model, dataloader):
    class_weights = [1.0, 10.0]
    optimizer = torch.optim.Adam(model.parameters(), lr=4e-8)
    criterion = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float32).to(device))
    model.train()
    total_loss = 0.0
    num_batches = len(dataloader)
    
    progress_bar = tqdm(enumerate(dataloader), total=num_batches, desc="Training")
    
    for batch_idx, (input_ids, attention_mask, labels) in progress_bar:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        
        # Update the progress bar description with current loss
        progress_bar.set_description(f"Training Loss: {loss.item():.4f}")
        
    return total_loss / num_batches


In [29]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def evaluate(model, dataloader):
    model.eval()
    predictions, true_labels = [], []

    with torch.no_grad():
        for input_ids, attention_mask, labels in dataloader:
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predicted_labels = torch.argmax(logits, dim=1)

            predictions.extend(predicted_labels.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    confusion = confusion_matrix(true_labels, predictions)

    # Calculate classification report
    report = classification_report(true_labels, predictions)

    print("Evaluation Metrics:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-Score: {f1}")
    print("Confusion Matrix:")
    print(confusion)
    print("Classification Report:")
    print(report)

    return accuracy, precision, recall, f1, confusion


In [30]:
def max_len_df(df):
    #maximum length of the sentence
    max_len = 0
    num_tokens = 0

    # For every sentence...
    for sent in df.data:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
        
        input_ids = tokenizer.encode(sent, add_special_tokens=True)
        num_tokens += (len(input_ids)-2)

        # Update the maximum sentence length.
        max_len = max(max_len, len(input_ids))

    return max_len

In [31]:
from sklearn.metrics import classification_report

def bert_classification(train_df, test_df):
    
    
        # Step 1: Preprocessing the Text Data
    #tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    max_len_train = max_len_df(train_df)
    max_len_test = max_len_df(test_df)
    
    max_len = max(max_len_train, max_len_test)

    # Step 2: Build the BERT Model
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    model.to(device)
    # Step 1: Preprocessing the Text Data
    train_texts = train_df['data'].tolist()
    train_labels = train_df['stereotype'].tolist()
    test_texts = test_df['data'].tolist()
    test_labels = test_df['stereotype'].tolist()

    train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt', )# max_length= 'Tokeniser max length'
    train_dataset = torch.utils.data.TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'],
                                                   torch.tensor(train_labels))
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

    test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors='pt')
    test_dataset = torch.utils.data.TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'],
                                                  torch.tensor(test_labels))
    test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

    # Step 4: Train the Model
    for epoch in range(15):
        loss = train(model, train_dataloader)
        print(f"Epoch {epoch + 1} - Average Loss: {loss}")

    # Step 5: Evaluate the Model
    evaluate(model, test_dataloader)

    # Step 6: Validate Model
    #val_texts = new_df['data'].tolist()
    #val_labels = new_df['stereotype'].tolist()

    #val_encodings = tokenizer(val_texts, truncation=True, padding=True, return_tensors='pt')
    #val_dataset = torch.utils.data.TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(val_labels))
    #val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

    #check = evaluate(model, val_dataloader)


In [32]:
train_data = []
test_data = []

for df in split_data_df:
    train_df = df[df['call_set'] == 'train']
    print(train_df.shape,'\n' ,train_df['stereotype'].value_counts())
    test_df = df[df['call_set'] == 'test']
    print(test_df.shape,'\n' ,test_df['stereotype'].value_counts())
    train_data.append(train_df)
    test_data.append(test_df)

(10156, 3) 
 0    8966
1    1190
Name: stereotype, dtype: int64
(3994, 3) 
 0    3837
1     157
Name: stereotype, dtype: int64
(5644, 3) 
 0    4596
1    1048
Name: stereotype, dtype: int64
(2091, 3) 
 0    1952
1     139
Name: stereotype, dtype: int64
(4170, 3) 
 0    3160
1    1010
Name: stereotype, dtype: int64
(1468, 3) 
 0    1356
1     112
Name: stereotype, dtype: int64
(1049, 3) 
 1    690
0    359
Name: stereotype, dtype: int64
(199, 3) 
 0    157
1     42
Name: stereotype, dtype: int64


In [30]:
stereo_list = ["Nurses are typically women, while doctors are usually men.",
"Men are more inclined towards careers in engineering and technology.",
"Women are more nurturing and better suited for caregiving roles.",
"Men are stronger and better equipped for physically demanding tasks.",
"Women are more emotional and sensitive compared to men.",
"Men are more assertive and confident in leadership positions.",
"Women are better at multitasking and managing household responsibilities.",
"Men are more interested in sports and physical activities than women.",
"Women are more fashion-conscious and spend more time on their appearance.",
"Men are better at handling finances and making investment decisions.",
"Women are emotional and irrational.",
"Men are strong and logical.",
"Women are better at nurturing and caring for children.",
"Men are more aggressive and competitive.",
"Women are not as good at math and science as men.",
"Men are not as good at expressing emotions as women.",
"Women are more talkative than men.",
"Men are better leaders than women.",
"Women are more concerned with their appearance than men.",
"Men are more interested in sports than women.",
"Individuals have diverse career interests and can pursue a wide range of professions.",
"Both men and women have the potential to excel in engineering and technology fields.",
"Skills and abilities are not determined by gender; individuals can excel in any caregiving role.",
"Tasks that require physical strength can be performed effectively by individuals of any gender.",
"Emotional expression varies among individuals and is not solely dependent on gender.",
"Leadership qualities such as assertiveness and confidence are found in individuals regardless of their gender.",
"Managing household responsibilities and multitasking are skills that can be developed by anyone.",
"Interest in sports and physical activities is not limited to a specific gender; it varies among individuals.",
"Individuals' fashion preferences and the time they spend on appearance vary irrespective of gender.",
"Financial management and investment decision-making abilities are not determined by gender.",
"Emotional expression and rationality are qualities present in individuals regardless of their gender.",
"Nurturing and caring for children are abilities that can be developed by anyone.",
"Aggression and competitiveness vary among individuals and are not exclusive to a particular gender.",
"Both men and women have the capability to excel in math and science fields.",
"Emotional expression is a trait that varies among individuals and is not limited to a specific gender.",
"Communication styles and talkativeness can differ among individuals, regardless of their gender.",
"Leadership qualities can be found in individuals of any gender.",
"Appearance concerns and personal grooming preferences vary among individuals, irrespective of their gender.",
"Interests in sports can vary among individuals and are not limited to a specific gender.",
"Individuals' interests, hobbies, and career choices are not determined by their gender."
]

In [31]:
#new_df = pd.DataFrame({'data':stereo_list, 'stereotype': [1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0]})
#new_df['data'] = new_df['data'].apply(clean_text)

In [32]:
#new_df

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [34]:
# Line By Line
bert_classification(train_data[0], test_data[0])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1 - Average Loss: 0.6196872398609251


Training Loss: 0.5527: 100%|██████████| 635/635 [01:07<00:00,  9.45it/s]


Epoch 2 - Average Loss: 0.5710891293728445


Training Loss: 0.4578: 100%|██████████| 635/635 [01:07<00:00,  9.46it/s]


...

Training Loss: 0.2756: 100%|██████████| 635/635 [01:06<00:00,  9.49it/s]


Epoch 14 - Average Loss: 0.31296180776489063


Training Loss: 0.2323: 100%|██████████| 635/635 [01:06<00:00,  9.53it/s]


Epoch 15 - Average Loss: 0.3077451207858371
Evaluation Metrics:
Accuracy: 0.9606910365548322
Precision: 0.0
Recall: 0.0
F1-Score: 0.0
Confusion Matrix:
[[3837    0]
 [ 157    0]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      3837
           1       0.00      0.00      0.00       157

    accuracy                           0.96      3994
   macro avg       0.48      0.50      0.49      3994
weighted avg       0.92      0.96      0.94      3994



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: Unde

In [31]:
bert_classification(train_data[1], test_data[1])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1 - Average Loss: 0.25750047797416326


Training Loss: 0.0180: 100%|██████████| 353/353 [01:30<00:00,  3.89it/s]


Epoch 2 - Average Loss: 0.09755763688412292


Training Loss: 0.4192: 100%|██████████| 353/353 [01:30<00:00,  3.89it/s]


Epoch 3 - Average Loss: 0.03596846083162297


Training Loss: 0.0003: 100%|██████████| 353/353 [01:30<00:00,  3.90it/s]


Epoch 4 - Average Loss: 0.025229964044468447


Training Loss: 0.0005: 100%|██████████| 353/353 [01:30<00:00,  3.90it/s]


Epoch 5 - Average Loss: 0.017869771456518396
Evaluation Metrics:
Accuracy: 0.93974175035868
Precision: 0.5414012738853503
Recall: 0.6115107913669064
F1-Score: 0.5743243243243243
Confusion Matrix:
[[1880   72]
 [  54   85]]
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.96      0.97      1952
           1       0.54      0.61      0.57       139

    accuracy                           0.94      2091
   macro avg       0.76      0.79      0.77      2091
weighted avg       0.94      0.94      0.94      2091



In [32]:
bert_classification(train_data[2], test_data[2])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1 - Average Loss: 0.2861665558441046


Training Loss: 0.0173: 100%|██████████| 261/261 [01:19<00:00,  3.27it/s]


Epoch 2 - Average Loss: 0.10584711340983965


Training Loss: 0.0973: 100%|██████████| 261/261 [01:19<00:00,  3.27it/s]


Epoch 3 - Average Loss: 0.04296139366206645


Training Loss: 0.0907: 100%|██████████| 261/261 [01:19<00:00,  3.27it/s]


Epoch 4 - Average Loss: 0.03413385733220747


Training Loss: 0.0008: 100%|██████████| 261/261 [01:19<00:00,  3.26it/s]


Epoch 5 - Average Loss: 0.016906100583596824
Evaluation Metrics:
Accuracy: 0.9305177111716622
Precision: 0.5416666666666666
Recall: 0.5803571428571429
F1-Score: 0.560344827586207
Confusion Matrix:
[[1301   55]
 [  47   65]]
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.96      0.96      1356
           1       0.54      0.58      0.56       112

    accuracy                           0.93      1468
   macro avg       0.75      0.77      0.76      1468
weighted avg       0.93      0.93      0.93      1468



In [33]:
bert_classification(train_data[3], test_data[3])

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch 1 - Average Loss: 0.4078547672340364


Training Loss: 0.0374: 100%|██████████| 66/66 [01:30<00:00,  1.37s/it]


Epoch 2 - Average Loss: 0.20030574472338863


Training Loss: 0.2037: 100%|██████████| 66/66 [01:30<00:00,  1.37s/it]


Epoch 3 - Average Loss: 0.11126794239901232


Training Loss: 0.0564: 100%|██████████| 66/66 [01:30<00:00,  1.37s/it]


Epoch 4 - Average Loss: 0.07773512501664685


Training Loss: 0.0052: 100%|██████████| 66/66 [01:30<00:00,  1.37s/it]


Epoch 5 - Average Loss: 0.03509459125978703
Evaluation Metrics:
Accuracy: 0.8291457286432161
Precision: 0.5714285714285714
Recall: 0.7619047619047619
F1-Score: 0.6530612244897959
Confusion Matrix:
[[133  24]
 [ 10  32]]
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.85      0.89       157
           1       0.57      0.76      0.65        42

    accuracy                           0.83       199
   macro avg       0.75      0.80      0.77       199
weighted avg       0.85      0.83      0.84       199



In [ ]:
split_data_df[1]['stereotype']

In [ ]:
print("Unique values in 'call_set' column:")
print(split_data_df[1]['call_set'].unique())


In [ ]:
print(split_data_df[1].keys())


In [ ]:
print("Values in 'data' column:")
print(split_data_df[1]['data'])

print("Values in 'stereotype' column:")
print(split_data_df[1]['stereotype'])

print("Values in 'call_set' column:")
print(split_data_df[1]['call_set'])


In [ ]:
import torch
from transformers import BertModel

def load_model(model_path):
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    model.to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()  # Set the model to evaluation mode
    
    return model

good_model = load_model('bert_model.pt')


In [ ]:
val_texts = new_df['data'].tolist()
val_labels = new_df['stereotype'].tolist()

val_encodings = tokenizer(val_texts, truncation=True, padding=True, return_tensors='pt')
val_dataset = torch.utils.data.TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], torch.tensor(val_labels))
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

check = evaluate(good_model, val_dataloader)